In [1]:
import cv2
import numpy as np
import pandas as pd
import ast
from glob import glob
# import matplotlib.pyplot as plt

In [42]:
df = pd.read_csv('Image_ROI.csv')

In [43]:
images = r".\images\2022_01_05_21_10\*"
img_paths = glob(images)

In [44]:
def display_image(img_name, img):
    cv2.namedWindow(img_name, cv2.WINDOW_NORMAL) # fit image to window
    cv2.imshow(img_name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [45]:
def roi(img, coords):
    x = int(img.shape[1])
    y = int(img.shape[0])
    if len(coords) < 4:
        print('minimum 4 coordinates required')
        return
    shape = np.array(coords) # shape of roi
    mask = np.zeros_like(img) # np array with zeros (of image dimension)

    # creates a polygon with the mask colour (blue), areas not in roi would be black (pixel is 0)
    cv2.fillPoly(mask, pts=np.int32([shape]), color=(255,255,255))

    # select ares where mask pixels are not zero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [46]:
for img_path in img_paths:
    img = cv2.imread(img_path)
    camera_id = int(img_path.split(
                '\\')[-1].split('.')[0].split('_')[0])
    if camera_id not in df['Camera_Id'].values:
        continue
    rows_df = df[df['Camera_Id'] == camera_id]
    for i in range(0, len(rows_df)):
        row = rows_df.iloc[i]
        coords = ast.literal_eval(row['ROI'])
        roi_img = roi(img, coords)
        display_image('ROI Image', roi_img)